In [ ]:
from binance.client import Client
import os
import telegram.ext
from telegram.ext import Updater, CommandHandler
from telegram.ext import MessageHandler, Filters
from decimal import Decimal

binance_api_key = os.environ.get("BINANCE_KEY")
binance_api_secret = os.environ.get("BINANCE_SECRET")
client = Client(binance_api_key, binance_api_secret)

telegram_token = os.environ.get("TELEGRAM_TOKEN")
updater = Updater(telegram_token, use_context=True)
bot = updater.bot
job = updater.job_queue

have_crypto = True

def getKlines (hours):
    history = []
    hours = (hours * 2) + 1
    print(hours)
    for kline in client.get_historical_klines_generator("ETHUSDT", 
                                                        Client.KLINE_INTERVAL_1HOUR, 
                                                        "{} hours ago UTC".format(hours)):
        history.append(float(kline[4]))
        
    del history[-1]
    return history

def rsi (history, rollingdays):
    gain = []
    loss = []
    for i in range(len(history)):
        if  i >= rollingdays:
            if history[i] - history[i-1] > 0:
                gain.append((history[i] - history[i-1]) / history[i-1])
            else: 
                loss.append((history[i] - history[i-1]) / history[i-1])
    avg_gain = sum(gain) / rollingdays
    avg_loss = abs(sum(loss) / rollingdays)
    print(avg_gain , avg_loss)
    return avg_gain/avg_loss         
                

def buyEth (update):
    balance = client.get_asset_balance(asset='USDT')
    trades = client.get_recent_trades(symbol='ETHUSDT')
    qty = (float(balance['free']))/float(trades[0]['price'])*0.998
    qty = '{:0.0{}f}'.format(qty, 6)
    qty = str(Decimal('{}'.format(qty)))
    market_price = client.get_recent_trades(symbol='ETHUSDT')
    market_price = round(float(market_price[0]['price']),6)
    order = client.order_market_buy(symbol='ETHUSDT', quantity=qty)
    
def sellEth (update):
    balance = client.get_asset_balance(asset='ETH')
    qty = (float(balance['free']))*0.998
    qty = '{:0.0{}f}'.format(qty, 6)
    qty = str(Decimal('{}'.format(qty)))
    market_price = client.get_recent_trades(symbol='ETHUSDT')
    market_price = round(float(market_price[0]['price']),6)
    order = client.order_market_sell(symbol='ETHUSDT', quantity=qty)
    
def rsiBot (update):
    global have_crypto
    RSI_index = 100 - (100 / (1 + rsi(getKlines(14), 14)))
    if RSI_index > 60:
        if have_crypto:
            sellEth(update)
            updater.bot.send_message(chat_id="@bebelere_balon", text= "Selling ETH, RSI: {0}".format(RSI_index))
            have_crypto = False
    elif RSI_index < 40:
        if have_crypto == False:
            buyEth(update)
            updater.bot.send_message(chat_id="@bebelere_balon", text= "Buying ETH, RSI: {0}".format(RSI_index))
            have_crypto = True
    else:
        updater.bot.send_message(chat_id="@bebelere_balon", text= "No order. RSI: {0}".format(RSI_index))

updater.job_queue.run_repeating(rsiBot, 3600, first=0)
updater.start_polling()
updater.idle()

In [ ]:
updater.bot.send_message(chat_id="@bebelere_balon", text= "No change.")

In [10]:
from binance.client import Client
import os

binance_api_key = os.environ.get("BINANCE_KEY")
binance_api_secret = os.environ.get("BINANCE_SECRET")
client = Client(binance_api_key, binance_api_secret)

def getKlines (hours):
    history = []
    hours = (hours * 2) + 1
    print(hours)
    for kline in client.get_historical_klines_generator("ETHUSDT", 
                                                        Client.KLINE_INTERVAL_1HOUR, 
                                                        "{} hours ago UTC".format(hours)):
        history.append(float(kline[4]))
        
    del history[-1]
    return history

def rsi (history, rollingdays):
    gain = []
    loss = []
    for i in range(len(history)):
        if  i >= rollingdays:
            if history[i] - history[i-1] > 0:
                gain.append((history[i] - history[i-1]) / history[i-1])
            else: 
                loss.append((history[i] - history[i-1]) / history[i-1])
    avg_gain = sum(gain) / rollingdays
    avg_loss = abs(sum(loss) / rollingdays)
    print(avg_gain , avg_loss)
    return avg_gain/avg_loss  
RSI_index = 100 - (100 / (1 + rsi(getKlines(14), 14)))
print(RSI_index)


29
0.0024725194914307557 0.0038167250168359957
39.31345789117929


In [79]:
from decimal import Decimal
from binance.client import Client
import os
import math

binance_api_key = os.environ.get("BINANCE_KEY")
binance_api_secret = os.environ.get("BINANCE_SECRET")
client = Client(binance_api_key, binance_api_secret)

balance = client.get_asset_balance(asset='ETH')
qty = (float(balance['free']))*0.998
qty = '{:0.0{}f}'.format(qty, 3)
qty = str(Decimal('{}'.format(qty)))
order = client.order_market_sell(symbol='ETHUSDT', quantity=qty)

{'symbol': 'ETHUSDT', 'orderId': 1037137524, 'orderListId': -1, 'clientOrderId': 'jAwj4f1b1ksZwQmoJdFU4q', 'transactTime': 1588805424793, 'price': '0.00000000', 'origQty': '0.49500000', 'executedQty': '0.49500000', 'cummulativeQuoteQty': '101.73240000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'fills': [{'price': '205.52000000', 'qty': '0.49500000', 'commission': '0.10173240', 'commissionAsset': 'USDT', 'tradeId': 145917273}]}


In [81]:
from decimal import Decimal
from binance.client import Client
import os

binance_api_key = os.environ.get("BINANCE_KEY")
binance_api_secret = os.environ.get("BINANCE_SECRET")
client = Client(binance_api_key, binance_api_secret)

balance = client.get_asset_balance(asset='USDT')['free']
balance = float(balance)
trades = client.get_recent_trades(symbol='ETHUSDT')
trades = float(trades[0]['price'])              
qty = (balance)/(trades)*0.998
qty = '{:0.0{}f}'.format(qty, 3)
qty = str(Decimal('{}'.format(qty)))
order = client.order_market_buy(symbol='ETHUSDT', quantity=qty)

101.72856898
trades 205.74 <class 'float'>
0.493
{'symbol': 'ETHUSDT', 'orderId': 1037146342, 'orderListId': -1, 'clientOrderId': 'TrzAt4dskzjKAIJ0tfhCPt', 'transactTime': 1588805534462, 'price': '0.00000000', 'origQty': '0.49300000', 'executedQty': '0.49300000', 'cummulativeQuoteQty': '101.45939780', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'fills': [{'price': '205.79000000', 'qty': '0.00022000', 'commission': '0.00000022', 'commissionAsset': 'ETH', 'tradeId': 145917718}, {'price': '205.80000000', 'qty': '0.49278000', 'commission': '0.00049278', 'commissionAsset': 'ETH', 'tradeId': 145917719}]}


In [5]:
balance = client.get_asset_balance(asset='ETH')
print(balance)

{'asset': 'ETH', 'free': '0.49595056', 'locked': '0.00000000'}


In [73]:
xxx = client.get_exchange_info()
type(xxx['symbols'])
for i in range(len(xxx['symbols'])):
    if xxx['symbols'][i]['symbol'] == 'ETHUSDT':
        print(xxx['symbols'][i]['filters'][5]['stepSize'])


0.0


In [35]:
print(xxx['symbols'][12]['filters'][5]['stepSize'])

0.00000000


In [ ]:
try:
    result = client.withdraw(
        asset='ETH',
        address='<eth_address>',
        amount=100)
except BinanceAPIException as e:
    print(e)
except BinanceWithdrawException as e:
    print(e)
else:
    print("Success")